In [29]:
import pandas as pd
import os

from utils.converter import h5_to_dict

In [30]:
OUTPUT_CSV_PATH = "data/songs.csv"
INPUT_FILE_PATH = "data/MillionSongSubset"

In [ ]:
rows = []

h5_to_dict("data/TRAXLZU12903D05F94.h5")

for root, _, files in os.walk(INPUT_FILE_PATH):
	for file in files:
		h5_path = os.path.join(root, file)
		try:
			song_data = h5_to_dict(h5_path)
			rows.append(song_data)
		except Exception as e :
			print(f"Error loading file {file}: {e}")

df = pd.DataFrame(rows)
df.to_csv(OUTPUT_CSV_PATH , index=False)

print(f"CSV created: {OUTPUT_CSV_PATH}")

CSV created: data/songs.csv


In [32]:
print(df.head(1))

                            analysis/bars_confidence  \
0  [0.643, 0.746, 0.722, 0.095, 0.091, 0.362, 0.4...   

                                 analysis/bars_start  \
0  [0.58521, 2.94247, 5.14371, 7.74554, 10.36149,...   

                           analysis/beats_confidence  \
0  [0.834, 0.851, 0.65, 0.635, 0.532, 0.753, 0.62...   

                                analysis/beats_start  \
0  [0.58521, 1.19196, 1.78893, 2.37813, 2.94247, ...   

                        analysis/sections_confidence  \
0  [1.0, 1.0, 0.218, 0.133, 0.384, 0.326, 0.373, ...   

                             analysis/sections_start  \
0  [0.0, 7.74554, 36.44331, 43.61667, 75.17954, 9...   

                        analysis/segments_confidence  \
0  [0.0, 1.0, 0.483, 0.137, 0.42, 1.0, 0.257, 1.0...   

                      analysis/segments_loudness_max  \
0  [-60.0, -31.646, -34.565, -38.407, -34.696, -2...   

                 analysis/segments_loudness_max_time  \
0  [0.0, 0.10929, 0.11044, 0.0844, 0.058